In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [83]:
train=pd.read_csv("/Users/hemanth/Downloads/ipl/deliveries.csv")

In [84]:
pd.set_option("max_columns",150)
pd.set_option("max_rows",150)

ind=train[train["is_super_over"]==1].index
train.drop(ind,axis=0,inplace=True)

In [85]:
ch1=train.groupby(["batting_team","over"],as_index=False).sum()[["batting_team","over","total_runs"]]
ch2=train.groupby(["batting_team","over"],as_index=False).count()[["batting_team","over","match_id"]]
ch2["match_id"]=ch2["match_id"]/6
ch1["total_runs"]=ch1["total_runs"]/ch2["match_id"]
ch1["batting_avg"]=ch1["total_runs"]

In [86]:
for i in range(len(ch1)):
    print i
    if ch1["batting_team"].iloc[i]==ch1["batting_team"].iloc[i-1]:
        ch1["batting_avg"].iloc[i]=ch1["batting_avg"].iloc[i-1]+ch1["total_runs"].iloc[i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [87]:
ch2=ch1[ch1["over"]==20]
ch2.columns=[["batting_team","sd","sd","avg"]]
test=pd.merge(ch1,ch2[["batting_team","avg"]],on=["batting_team"],how="left")
test["avg"]=test["avg"]-test["batting_avg"]
test.columns=[["batting_team","over","df","df","bat_runs_avg"]]
test=test[["batting_team","over","bat_runs_avg"]]

In [88]:
ch1=train.groupby(["bowling_team","over"],as_index=False).sum()[["bowling_team","over","total_runs"]]
ch2=train.groupby(["bowling_team","over"],as_index=False).count()[["bowling_team","over","match_id"]]
ch2["match_id"]=ch2["match_id"]/6
ch1["total_runs"]=ch1["total_runs"]/ch2["match_id"]
ch1["batting_avg"]=ch1["total_runs"]

In [89]:
for i in range(len(ch1)):
    print i
    if ch1["bowling_team"].iloc[i]==ch1["bowling_team"].iloc[i-1]:
        ch1["batting_avg"].iloc[i]=ch1["batting_avg"].iloc[i-1]+ch1["total_runs"].iloc[i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [90]:
ch2=ch1[ch1["over"]==20]
ch2.columns=[["bowling_team","sd","sd","avg"]]
test2=pd.merge(ch1,ch2[["bowling_team","avg"]],on=["bowling_team"],how="left")
test2["avg"]=test2["avg"]-test2["batting_avg"]
test2.columns=[["bowling_team","over","df","df","bowl_runs_avg"]]
test2=test2[["bowling_team","over","bowl_runs_avg"]]

In [91]:
test.columns=[["team","over","bat_runs_avg"]]
test2.columns=[["team","over","bowl_runs_avg"]]
test=pd.merge(test,test2,on=["team","over"],how="left")

In [92]:
check=train[~train["player_dismissed"].isnull()]


In [93]:
ch1=check.groupby(["batting_team","over"],as_index=False).count()[["batting_team","over","total_runs"]]
ch2=train.groupby(["batting_team","over"],as_index=False).count()[["batting_team","over","match_id"]]
ch2["match_id"]=ch2["match_id"]/6
ch1["total_runs"]=ch1["total_runs"]/ch2["match_id"]
ch1["batting_avg"]=ch1["total_runs"]

In [94]:
for i in range(len(ch1)):
    print i
    if ch1["batting_team"].iloc[i]==ch1["batting_team"].iloc[i-1]:
        ch1["batting_avg"].iloc[i]=ch1["batting_avg"].iloc[i-1]+ch1["total_runs"].iloc[i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [95]:
ch2=ch1[ch1["over"]==20]
ch2.columns=[["batting_team","sd","sd","avg"]]
test2=pd.merge(ch1,ch2[["batting_team","avg"]],on=["batting_team"],how="left")
test2["avg"]=test2["avg"]-test2["batting_avg"]
test2.columns=[["team","over","df","df","bat_wickets_avg"]]
test2=test2[["team","over","bat_wickets_avg"]]

In [101]:
test=pd.merge(test,test2[["team","bat_wickets_avg"]],on=["team"],how="left")

In [102]:
ch1=check.groupby(["bowling_team","over"],as_index=False).count()[["bowling_team","over","total_runs"]]
ch2=train.groupby(["bowling_team","over"],as_index=False).count()[["bowling_team","over","match_id"]]
ch2["match_id"]=ch2["match_id"]/6
ch1["total_runs"]=ch1["total_runs"]/ch2["match_id"]
ch1["batting_avg"]=ch1["total_runs"]

In [103]:
for i in range(len(ch1)):
    print i
    if ch1["bowling_team"].iloc[i]==ch1["bowling_team"].iloc[i-1]:
        ch1["batting_avg"].iloc[i]=ch1["batting_avg"].iloc[i-1]+ch1["total_runs"].iloc[i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [104]:
ch2=ch1[ch1["over"]==20]
ch2.columns=[["bowling_team","sd","sd","avg"]]
test2=pd.merge(ch1,ch2[["bowling_team","avg"]],on=["bowling_team"],how="left")
test2["avg"]=test2["avg"]-test2["batting_avg"]
test2.columns=[["team","over","df","df","bowl_wickets_avg"]]
test2=test2[["team","over","bowl_wickets_avg"]]

In [105]:
test=pd.merge(test,test2[["team","bowl_wickets_avg"]],on=["team"],how="left")

In [106]:
test

,team,over,bat_runs_avg,bowl_runs_avg,bat_wickets_avg,bowl_wickets_avg
0,Chennai Super Kings,1,155.336831,147.984656,5.147297,6.390124
1,Chennai Super Kings,1,155.336831,147.984656,5.147297,6.186240
2,Chennai Super Kings,1,155.336831,147.984656,5.147297,5.893200
3,Chennai Super Kings,1,155.336831,147.984656,5.147297,5.663853
4,Chennai Super Kings,1,155.336831,147.984656,5.147297,5.374249
5,Chennai Super Kings,1,155.336831,147.984656,5.147297,5.083566
6,Chennai Super Kings,1,155.336831,147.984656,5.147297,4.885764
7,Chennai Super Kings,1,155.336831,147.984656,5.147297,4.709293
8,Chennai Super Kings,1,155.336831,147.984656,5.147297,4.472840
9,Chennai Super Kings,1,155.336831,147.984656,5.147297,4.232239
